6 May 2024
## Working with databases in Python

In [3]:
#%pip install duckdb
import duckdb

Create a connection and a cursor

In [4]:
conn = duckdb.connect("database.db")
conn

In [5]:
cur = conn.cursor()

In [9]:
cur.execute("SELECT * FROM Site LIMIT 5")

Get the results in three different ways
1. All at once

In [10]:
cur.fetchall()

[('barr',
  'Barrow',
  'Alaska, USA',
  71.30000305175781,
  -156.60000610351562,
  220.39999389648438),
 ('burn',
  'Burntpoint Creek',
  'Ontario, Canada',
  55.20000076293945,
  -84.30000305175781,
  63.0),
 ('bylo',
  'Bylot Island',
  'Nunavut, Canada',
  73.19999694824219,
  -80.0,
  723.5999755859375),
 ('cakr',
  'Cape Krusenstern',
  'Alaska, USA',
  67.0999984741211,
  -163.5,
  54.099998474121094),
 ('cari',
  'Canning River Delta',
  'Alaska, USA',
  70.0999984741211,
  -145.8000030517578,
  722.0)]

In [13]:
cur.execute("SELECT Nest_ID FROM BIrd_nests LIMIT 10")
[t[0] for t in cur.fetchall()]

['14HPE1',
 '11eaba',
 '11eabaagc01',
 '11eabaagv01',
 '11eababbc02',
 '11eababsv01',
 '11eabaduh01',
 '11eabaduv01',
 '11eabarpc01',
 '11eabarpc02']

2. Get the one result, or the next result

In [15]:
cur.execute("SELECT COUNT(*) FROM Bird_nests")
cur.fetchone()[0] # get one result

1547

3. Using and in iterator - but DuckDB doesn't support iterators

In [ ]:
# this would would with sqlite or many other databases
cur.execute("SELECT Nest_ID FROM Bird_nests LIMIT 10")
for row in cur:
    print(f"got {row[0]}") # this does not run

In [16]:
# a work around
cur.execute("SELECT Nest_ID FROM Bird_nests LIMIT 10")
while True:
    row = cur.fetchone()
    if row == None:
        break
    # do something with row
    print(f"got nest ID {row[0]}")

got nest ID 14HPE1
got nest ID 11eaba
got nest ID 11eabaagc01
got nest ID 11eabaagv01
got nest ID 11eababbc02
got nest ID 11eababsv01
got nest ID 11eabaduh01
got nest ID 11eabaduv01
got nest ID 11eabarpc01
got nest ID 11eabarpc02


Can do other things than SELECT

In [17]:
# make a temp table
cur.execute("""
            CREATE TEMP TABLE temp_table AS
            SELECT * FROM Bird_nests LIMIT 10
            """)

cur.execute("SELECT * FROM temp_table")
cur.fetchall()


[('b14.6',
  2014,
  'chur',
  '14HPE1',
  'sepl',
  'vloverti',
  datetime.date(2014, 6, 14),
  None,
  3,
  None,
  None),
 ('b11.7',
  2011,
  'eaba',
  '11eaba',
  'wrsa',
  'bhill',
  datetime.date(2011, 7, 10),
  'searcher',
  4,
  None,
  None),
 ('b11.6',
  2011,
  'eaba',
  '11eabaagc01',
  'amgp',
  'dkessler',
  datetime.date(2011, 6, 24),
  'searcher',
  4,
  6.0,
  'float'),
 ('b11.6',
  2011,
  'eaba',
  '11eabaagv01',
  'amgp',
  'dkessler',
  datetime.date(2011, 6, 25),
  'searcher',
  3,
  3.0,
  'float'),
 ('b11.6',
  2011,
  'eaba',
  '11eababbc02',
  'bbpl',
  'dkessler',
  datetime.date(2011, 6, 24),
  'searcher',
  4,
  4.0,
  'float'),
 ('b11.7',
  2011,
  'eaba',
  '11eababsv01',
  'wrsa',
  'bhill',
  datetime.date(2011, 7, 7),
  'searcher',
  4,
  2.0,
  'float'),
 ('b11.6',
  2011,
  'eaba',
  '11eabaduh01',
  'dunl',
  'dkessler',
  datetime.date(2011, 6, 28),
  'searcher',
  3,
  2.0,
  'float'),
 ('b11.6',
  2011,
  'eaba',
  '11eabaduv01',
  'dunl',
  'dk

A note on fragility

For example:
```
INSERT INTO Site VALUES ("abcd", "Foo", 35.7, -119.5, "?") 
```
- order of values has to be in the order of the columns, but how do we know the order of the columns?

A less fragile way:
```
INSERT INTO Site (Code, Site_name, Latitude, Longitude, Something_else)
    VALUES ("abcd", "Foo", 35.7, -119.5, "?") 
```

`SELECT *` is also fragile

From our `cur.fetchall()`, we don't see the column names.
A more robust way of coding this:


In [19]:
cur.execute("SELECT Code, Site_name, Latitude, Longitude FROM Site LIMIT 3")
cur.fetchall()

[('barr', 'Barrow', 71.30000305175781, -156.60000610351562),
 ('burn', 'Burntpoint Creek', 55.20000076293945, -84.30000305175781),
 ('bylo', 'Bylot Island', 73.19999694824219, -80.0)]

- If someone inserts a new column and you use `SELECT *`, you might get something different than expected. This is a more explicit way

#### An extended example:
**Question:** How many nests do we have for each species?

**Approach:** Get all species, then execute count query for each species

In [22]:
query = """
SELECT COUNT(*) FROM Bird_nests
WHERE Species = '%s'
"""

cur.execute("SELECT Code FROM Species LIMIT 3")
for row in cur.fetchall():
    code = row[0]
    prepared_query = query % code
    #print(prepared_query)
    cur2 = conn.cursor()
    cur2.execute(prepared_query)
    print(f"Species {code} has {cur2.fetchone()[0]} nests")

Species agsq has 0 nests
Species amcr has 0 nests
Species amgp has 29 nests


The above Python interpolation is dangerous and has caused many database hacks. There is a better way:

In [23]:
query = """
SELECT COUNT(*) FROM Bird_nests
WHERE Species = ?
"""

cur.execute("SELECT Code FROM Species LIMIT 3")
for row in cur.fetchall():
    code = row[0]
    #prepared_query = query % code
    #print(prepared_query)
    cur2 = conn.cursor()
    cur2.execute(query, [code]) # <- add argument
    print(f"Species {code} has {cur2.fetchone()[0]} nests")

Species agsq has 0 nests
Species amcr has 0 nests
Species amgp has 29 nests


We're doing the same thing, but we give it a second argument, and it will add that argument where there is a "?" in the query.

Let's illustrate the danger with a different example:

In [28]:
abbrev = "CO"
name = "Conan O'Brien"
cur.execute("""
            INSERT INTO Personnel (Abbreviation, Name)
            VALUES ('%s', '%s')
            """ % (abbrev, name)
            )

cur.execute("SELECT * FROM Personnel")
cur.fetchall()[-3:]

ParserException: Parser Error: syntax error at or near "Brien"

Problem with having and apostrophe in the name, but using the '?' method solves the issue:

In [30]:
abbrev = "CO"
name = "Conan O'Brien"
cur.execute("""
            INSERT INTO Personnel (Abbreviation, Name)
            VALUES (?, ?)
            """, (abbrev, name)
            )

cur.execute("SELECT * FROM Personnel")
cur.fetchall()[-3:]

[('mcorrell', 'Maureen Correll'),
 ('TS', 'Taylor Swift'),
 ('CO', "Conan O'Brien")]